In [40]:
from pathlib import Path
import numpy as np
from enum import Enum

In [36]:
def parse_map(lines):
    width = max(len(l) for l in lines)
    M = [list(l + ' ' * (width-len(l))) for l in lines]
    return np.array(M)
        
def parse_inst(insts):
    cur = ''
    out = []
    for c in insts:
        if c in ['L','R']:
            if len(cur) > 0:
                out.extend([int(cur),c])
            else:
                out.append(c)
            cur = ''
        else:
            cur = cur + c
    return out

def read(prefix='data'):
    M, I = Path(f'{prefix}/22.txt').read_text().rstrip().split('\n\n')
    return parse_map(M.rstrip().split('\n')), parse_inst(I)



In [37]:
M, I = read('test')

In [38]:
disp(M)

        ...#    
        .#..    
        #...    
        ....    
...#.......#    
........#...    
..#....#....    
..........#.    
        ...#....
        .....#..
        .#......
        ......#.


In [39]:
I

[10, 'R', 5, 'L', 5, 'R', 10, 'L', 4, 'R', 5, 'L']

In [65]:
class Facing(Enum):
    right = 0
    down = 1
    left = 2
    up = 3

    def turnR(self):
        val = (self.value +1) % 4
        return Facing(val)

    def turnL(self):
        val = (self.value +1) % 4
        return Facing(val)

    def turn(self, d):
        if d == 'L':
            return self.turnL()
        elif d == 'R':
            return self.turnR()
        else:
            raise Exception(f'unknown direction {d=}')

In [97]:
class Map:
    def __init__(self, M):
        self._M = M
        self.x = list(M[0]).index('.')
        self.y = 0
        self.facing = Facing.right
        self.__set_boundary()
    
    def __set_boundary(self):
        self.edgeL, self.edgeR = [], []
        for i in range(self._M.shape[0]):
            onmap = np.where(self._M[i] != ' ')[0]
            self.edgeL.append(onmap[0])
            self.edgeR.append(onmap[-1])

        self.edgeT, self.edgeB = [], []
        for i in range(self._M.shape[1]):
            onmap = np.where(self._M[:,i] != ' ')[0]
            self.edgeT.append(onmap[0])
            self.edgeB.append(onmap[-1])


    def move(self, inst):
        if inst in ['L', 'R']:
            self.facing = self.facing.turn(inst)

        inst = int(inst)
        self.forward(inst)

    def disp(self):
        print(*[''.join(r) for r in self._M], sep='\n')

    def password(self):
        return ((self.y+1) * 1000)  + ((self.x+1) * 4) + self.facing.value

In [108]:
M, I = read('test')
map = Map(M)
map.disp()

        ...#    
        .#..    
        #...    
        ....    
...#.......#    
........#...    
..#....#....    
..........#.    
        ...#....
        .....#..
        .#......
        ......#.


In [121]:
map.edgeB

[7, 7, 7, 7, 7, 7, 7, 7, 11, 11, 11, 11, 11, 11, 11, 11]

In [120]:
M[8,-1]

'.'